# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import sqlite3
import pandas as pd
import numpy as np
import scipy as sp
import re
from sqlalchemy import create_engine
import joblib

import nltk
from nltk.tokenize import word_tokenize
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import stopwords

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression, SGDClassifier

nltk.download("punkt")
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\phewv\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\phewv\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\phewv\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\phewv\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [2]:
# load data from database
engine = create_engine('sqlite:///DisasterResponse.db')
df = pd.read_sql_table('DisasterResponse',engine)
X = df.message.values
y = df.drop(columns=df.columns[0:4],axis=1)


### 2. Write a tokenization function to process your text data

In [6]:
def tokenize(text):
    
    text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower())
    
    tokens = word_tokenize(text)

    tokens = [WordNetLemmatizer().lemmatize(w).strip() for w in tokens if w not in stopwords.words("english")]

    return tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [7]:
pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(estimator=MultinomialNB()))
]) 

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=.3)
pipeline.fit(X_train , y_train)


c:\Users\phewv\anaconda3\envs\Env_1\lib\site-packages\sklearn\feature_extraction\text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


Pipeline(steps=[('vect',
                 CountVectorizer(tokenizer=<function tokenize at 0x000001E1D3ADF520>)),
                ('tfidf', TfidfTransformer()),
                ('clf', MultiOutputClassifier(estimator=MultinomialNB()))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [9]:
y_pred = pipeline.predict(X_test)

In [10]:
for col in range(y_pred.shape[1]):

    print(classification_report(y_test.iloc[:,col],y_pred[:,col],labels=np.unique(y_pred[:,col])))
    print(col)


              precision    recall  f1-score   support

           0       0.81      0.10      0.17      1841
           1       0.77      0.99      0.87      5956

   micro avg       0.77      0.78      0.78      7797
   macro avg       0.79      0.54      0.52      7797
weighted avg       0.78      0.78      0.70      7797

0
              precision    recall  f1-score   support

           0       0.87      0.99      0.93      6573
           1       0.86      0.22      0.35      1292

    accuracy                           0.87      7865
   macro avg       0.86      0.61      0.64      7865
weighted avg       0.86      0.87      0.83      7865

1
              precision    recall  f1-score   support

           0       0.99      1.00      1.00      7821

   micro avg       0.99      1.00      1.00      7821
   macro avg       0.99      1.00      1.00      7821
weighted avg       0.99      1.00      1.00      7821

2
              precision    recall  f1-score   support

           0

### 6. Improve your model
Use grid search to find better parameters. 

In [11]:
pipeline.get_params()

{'memory': None,
 'steps': [('vect',
   CountVectorizer(tokenizer=<function tokenize at 0x000001E1D3ADF520>)),
  ('tfidf', TfidfTransformer()),
  ('clf', MultiOutputClassifier(estimator=MultinomialNB()))],
 'verbose': False,
 'vect': CountVectorizer(tokenizer=<function tokenize at 0x000001E1D3ADF520>),
 'tfidf': TfidfTransformer(),
 'clf': MultiOutputClassifier(estimator=MultinomialNB()),
 'vect__analyzer': 'word',
 'vect__binary': False,
 'vect__decode_error': 'strict',
 'vect__dtype': numpy.int64,
 'vect__encoding': 'utf-8',
 'vect__input': 'content',
 'vect__lowercase': True,
 'vect__max_df': 1.0,
 'vect__max_features': None,
 'vect__min_df': 1,
 'vect__ngram_range': (1, 1),
 'vect__preprocessor': None,
 'vect__stop_words': None,
 'vect__strip_accents': None,
 'vect__token_pattern': '(?u)\\b\\w\\w+\\b',
 'vect__tokenizer': <function __main__.tokenize(text)>,
 'vect__vocabulary': None,
 'tfidf__norm': 'l2',
 'tfidf__smooth_idf': True,
 'tfidf__sublinear_tf': False,
 'tfidf__use_idf':

In [12]:
parameters = {

    'clf__estimator__alpha': [0.1, 0.5]
       
}

cv = GridSearchCV(pipeline,param_grid=parameters)

cv.fit(X_train,y_train)

c:\Users\phewv\anaconda3\envs\Env_1\lib\site-packages\sklearn\feature_extraction\text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


KeyboardInterrupt: 

In [ ]:
print("\nBest Parameters:", cv.best_params_)


Best Parameters: {'clf__estimator__alpha': 0.1}


### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [ ]:
pipeline_tuned = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(estimator=MultinomialNB(alpha=0.1)))
])

pipeline_tuned.fit(X_train , y_train)
y_pred_tuned = pipeline_tuned.predict(X_test)

for col in range(y_pred_tuned.shape[1]):

    print(classification_report(y_test.iloc[:,col],y_pred_tuned[:,col],labels=np.unique(y_pred_tuned[:,col])))
    

c:\Users\phewv\anaconda3\envs\Env_1\lib\site-packages\sklearn\feature_extraction\text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


              precision    recall  f1-score   support

           0       0.71      0.37      0.49      1874
           1       0.82      0.95      0.88      5934
           2       0.74      0.30      0.42        57

    accuracy                           0.81      7865
   macro avg       0.76      0.54      0.60      7865
weighted avg       0.80      0.81      0.79      7865

0
              precision    recall  f1-score   support

           0       0.90      0.95      0.93      6478
           1       0.71      0.53      0.61      1387

    accuracy                           0.88      7865
   macro avg       0.81      0.74      0.77      7865
weighted avg       0.87      0.88      0.87      7865

1
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      7831
           1       0.00      0.00      0.00        34

    accuracy                           1.00      7865
   macro avg       0.50      0.50      0.50      7865
weighted avg    

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [13]:

#See which column has all 0 values
print(y_train.sum())

#remove the column that has all 0 values
y_train_test = y_train.drop(labels=['child_alone'],axis=1)
y_train_test.columns


related                   14190
request                    3182
offer                        74
aid_related                7642
medical_help               1456
medical_products            932
search_and_rescue           519
security                    338
military                    604
child_alone                   0
water                      1209
food                       2062
shelter                    1629
clothing                    288
money                       419
missing_people              202
refugees                    609
death                       839
other_aid                  2436
infrastructure_related     1196
transport                   853
buildings                   951
electricity                 381
tools                       114
hospitals                   192
shops                        78
aid_centers                 223
other_infrastructure        809
weather_related            5084
floods                     1501
storm                      1687
fire    

Index(['related', 'request', 'offer', 'aid_related', 'medical_help',
       'medical_products', 'search_and_rescue', 'security', 'military',
       'water', 'food', 'shelter', 'clothing', 'money', 'missing_people',
       'refugees', 'death', 'other_aid', 'infrastructure_related', 'transport',
       'buildings', 'electricity', 'tools', 'hospitals', 'shops',
       'aid_centers', 'other_infrastructure', 'weather_related', 'floods',
       'storm', 'fire', 'earthquake', 'cold', 'other_weather',
       'direct_report'],
      dtype='object')

In [ ]:
df.related.unique()

array([1, 0, 2], dtype=int64)

In [15]:
pipeline_log = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(estimator=LogisticRegression()))
]) 

pipeline_log.fit(X_train , y_train_test)
y_pred_SVC = pipeline_log.predict(X_test)

for col in range(y_pred_SVC.shape[1]):

    print(classification_report(y_test.iloc[:,col],y_pred_SVC[:,col]))

c:\Users\phewv\anaconda3\envs\Env_1\lib\site-packages\sklearn\feature_extraction\text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
c:\Users\phewv\anaconda3\envs\Env_1\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\phewv\anaconda3\envs\Env_1\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\phewv\anaconda3\envs\Env_1\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are i

              precision    recall  f1-score   support

           0       0.72      0.45      0.55      1841
           1       0.84      0.95      0.89      5956
           2       0.00      0.00      0.00        68

    accuracy                           0.82      7865
   macro avg       0.52      0.46      0.48      7865
weighted avg       0.80      0.82      0.80      7865

              precision    recall  f1-score   support

           0       0.91      0.98      0.94      6573
           1       0.81      0.52      0.63      1292

    accuracy                           0.90      7865
   macro avg       0.86      0.75      0.79      7865
weighted avg       0.89      0.90      0.89      7865

              precision    recall  f1-score   support

           0       0.99      1.00      1.00      7821
           1       0.00      0.00      0.00        44

    accuracy                           0.99      7865
   macro avg       0.50      0.50      0.50      7865
weighted avg       0

c:\Users\phewv\anaconda3\envs\Env_1\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\phewv\anaconda3\envs\Env_1\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\phewv\anaconda3\envs\Env_1\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\phewv\anacond

In [ ]:
pipeline_SVC = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(estimator=SGDClassifier(loss='hinge', penalty='l2',alpha=1e-3, random_state=42, max_iter=5, tol=None)))
]) 

pipeline_SVC.fit(X_train , y_train_test)
y_pred_SVC = pipeline_SVC.predict(X_test)

for col in range(y_pred_SVC.shape[1]):

    print(classification_report(y_test.iloc[:,col],y_pred_SVC[:,col]))

c:\Users\phewv\anaconda3\envs\Env_1\lib\site-packages\sklearn\feature_extraction\text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
c:\Users\phewv\anaconda3\envs\Env_1\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\phewv\anaconda3\envs\Env_1\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\phewv\anaconda3\envs\Env_1\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are i

              precision    recall  f1-score   support

           0       0.80      0.07      0.12      1874
           1       0.77      0.99      0.87      5934
           2       0.00      0.00      0.00        57

    accuracy                           0.77      7865
   macro avg       0.52      0.35      0.33      7865
weighted avg       0.77      0.77      0.68      7865

              precision    recall  f1-score   support

           0       0.87      0.99      0.93      6478
           1       0.91      0.33      0.48      1387

    accuracy                           0.88      7865
   macro avg       0.89      0.66      0.71      7865
weighted avg       0.88      0.88      0.85      7865

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      7831
           1       0.00      0.00      0.00        34

    accuracy                           1.00      7865
   macro avg       0.50      0.50      0.50      7865
weighted avg       0

c:\Users\phewv\anaconda3\envs\Env_1\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\phewv\anaconda3\envs\Env_1\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\phewv\anaconda3\envs\Env_1\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\phewv\anacond

              precision    recall  f1-score   support

           0       0.95      1.00      0.97      7468
           1       0.00      0.00      0.00       397

    accuracy                           0.95      7865
   macro avg       0.47      0.50      0.49      7865
weighted avg       0.90      0.95      0.92      7865

              precision    recall  f1-score   support

           0       0.97      1.00      0.99      7642
           1       0.00      0.00      0.00       223

    accuracy                           0.97      7865
   macro avg       0.49      0.50      0.49      7865
weighted avg       0.94      0.97      0.96      7865

              precision    recall  f1-score   support

           0       0.98      1.00      0.99      7730
           1       0.00      0.00      0.00       135

    accuracy                           0.98      7865
   macro avg       0.49      0.50      0.50      7865
weighted avg       0.97      0.98      0.97      7865

              preci

c:\Users\phewv\anaconda3\envs\Env_1\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\phewv\anaconda3\envs\Env_1\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\phewv\anaconda3\envs\Env_1\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\phewv\anacond

              precision    recall  f1-score   support

           0       0.87      1.00      0.93      6843
           1       0.00      0.00      0.00      1022

    accuracy                           0.87      7865
   macro avg       0.44      0.50      0.47      7865
weighted avg       0.76      0.87      0.81      7865

              precision    recall  f1-score   support

           0       0.94      1.00      0.97      7370
           1       0.00      0.00      0.00       495

    accuracy                           0.94      7865
   macro avg       0.47      0.50      0.48      7865
weighted avg       0.88      0.94      0.91      7865

              precision    recall  f1-score   support

           0       0.95      1.00      0.98      7504
           1       0.00      0.00      0.00       361

    accuracy                           0.95      7865
   macro avg       0.48      0.50      0.49      7865
weighted avg       0.91      0.95      0.93      7865

              preci

c:\Users\phewv\anaconda3\envs\Env_1\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\phewv\anaconda3\envs\Env_1\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\phewv\anaconda3\envs\Env_1\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\phewv\anacond

              precision    recall  f1-score   support

           0       0.99      1.00      0.99      7773
           1       0.00      0.00      0.00        92

    accuracy                           0.99      7865
   macro avg       0.49      0.50      0.50      7865
weighted avg       0.98      0.99      0.98      7865

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      7837
           1       0.00      0.00      0.00        28

    accuracy                           1.00      7865
   macro avg       0.50      0.50      0.50      7865
weighted avg       0.99      1.00      0.99      7865

              precision    recall  f1-score   support

           0       0.99      1.00      0.99      7776
           1       0.00      0.00      0.00        89

    accuracy                           0.99      7865
   macro avg       0.49      0.50      0.50      7865
weighted avg       0.98      0.99      0.98      7865

              preci

c:\Users\phewv\anaconda3\envs\Env_1\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\phewv\anaconda3\envs\Env_1\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\phewv\anaconda3\envs\Env_1\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\phewv\anacond

              precision    recall  f1-score   support

           0       0.90      1.00      0.95      7102
           1       0.00      0.00      0.00       763

    accuracy                           0.90      7865
   macro avg       0.45      0.50      0.47      7865
weighted avg       0.82      0.90      0.86      7865

              precision    recall  f1-score   support

           0       0.99      0.95      0.97      7782
           1       0.01      0.04      0.01        83

    accuracy                           0.94      7865
   macro avg       0.50      0.49      0.49      7865
weighted avg       0.98      0.94      0.96      7865

              precision    recall  f1-score   support

           0       0.91      1.00      0.95      7154
           1       0.00      0.00      0.00       711

    accuracy                           0.91      7865
   macro avg       0.45      0.50      0.48      7865
weighted avg       0.83      0.91      0.87      7865

              preci

### 9. Export your model as a pickle file

In [16]:
joblib.dump(pipeline_log, 'Pipeline_Log.pkl')

['Pipeline_Log.pkl']

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.